# Convolutional Neural Networks for Dogs and Cats

## Preparation (40 points total)
#### [10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).  

In [2]:
%%time
from PIL import Image
import pandas as pd
import numpy as np

image_size = (40, 40)
total_image_size = image_size[0]**2
image_size3 = (1, 40, 40)
X = []
y = []
label = [0, 1] #cat == 0, dog == 1
for i in range(0, 12000):
    image = Image.open('train/cat.' + str(i) +'.jpg')
    image = image.resize(image_size)
    image = image.convert('L')
    image_array = np.array(image).reshape(total_image_size,)
    X.append(image_array)
    y.append(0)
    
for i in range(0, 12000):
    image = Image.open('train/dog.' + str(i) +'.jpg')
    image = image.resize(image_size)
    image = image.convert('L')
    image_array = np.array(image).reshape(total_image_size,)
    X.append(image_array)
    y.append(1)

X = np.asarray(X)
y = np.asarray(y)

Wall time: 55 s


In [3]:
print(X.shape)
print(y.shape)

X = X.astype(np.float32) / 255.0 # normalize

(24000, 1600)
(24000,)


#### [15 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

#### [15 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice.

In [46]:
import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

y_ohe_train = keras.utils.to_categorical(y_train, 2)
y_ohe_test = keras.utils.to_categorical(y_test, 2)


## Modeling (50 points total)
#### [20 points] Create a convolutional neural network to use on your data using tensorflow. 

In [5]:
%%time
cnn2 = Sequential()
cnn2.add(Reshape(image_size3, input_shape=(1, total_image_size)))
cnn2.add(Conv2D(filters=16, kernel_size=(2, 2), padding='same'))
cnn2.add(Activation('relu'))
cnn2.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))


cnn2.add(Flatten())
cnn2.add(Dense(2))
cnn2.add(Activation('softmax'))

cnn2.compile(loss='mean_squared_logarithmic_error',
            optimizer='adam',
            metrics=['accuracy'])

cnn2.fit(np.expand_dims(X_train, axis=1), y_ohe_train, batch_size=32, epochs=50, shuffle=True, verbose=0)

Wall time: 1min 42s


In [6]:
from sklearn import metrics as mt

yhat2 = np.argmax(cnn2.predict(np.expand_dims(X_test, axis=1)), axis=1)
print(yhat2.shape)
acc2 = mt.accuracy_score(y_test, yhat2)
acc2

(4800,)


0.65937500000000004

#### [20 points] Investigate at least two different deep network architectures. Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab. Visualize the results of the CNNs. 

In [8]:
%%time
cnn = Sequential()

cnn.add(Reshape(image_size3, input_shape=(1,total_image_size)))
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

cnn.add(Dropout(.25))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Dropout(.25))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))


cnn.add(Flatten())
cnn.add(Dense(2))
cnn.add(Activation('softmax'))

cnn.compile(loss='mean_squared_logarithmic_error',
            optimizer='adam',
            metrics=['accuracy'])

cnn.fit(np.expand_dims(X_train, axis=1), y_ohe_train, batch_size=32, epochs=50, shuffle=True, verbose=1)

Epoch 1/50
19200/19200 [==============================] - 19s - loss: 0.1216 - acc: 0.5450    
Epoch 2/50
19200/19200 [==============================] - 18s - loss: 0.1178 - acc: 0.5886    
Epoch 3/50
19200/19200 [==============================] - 19s - loss: 0.1159 - acc: 0.6066    
Epoch 4/50
19200/19200 [==============================] - 18s - loss: 0.1146 - acc: 0.6119    
Epoch 5/50
19200/19200 [==============================] - 18s - loss: 0.1131 - acc: 0.6195    
Epoch 6/50
19200/19200 [==============================] - 18s - loss: 0.1121 - acc: 0.6254    
Epoch 7/50
19200/19200 [==============================] - 19s - loss: 0.1117 - acc: 0.6276    
Epoch 8/50
19200/19200 [==============================] - 18s - loss: 0.1117 - acc: 0.6294    
Epoch 9/50
19200/19200 [==============================] - 18s - loss: 0.1103 - acc: 0.6359    
Epoch 10/50
19200/19200 [==============================] - 18s - loss: 0.1101 - acc: 0.6356    
Epoch 11/50
19200/19200 [========================

In [68]:
yhat = np.argmax(cnn.predict(np.expand_dims(X_test, axis=1)), axis=1)
yhat_probs = np.argmax(cnn.predict_proba(np.expand_dims(X_test, axis=1)))
acc = mt.accuracy_score(y_test, yhat)
acc

SyntaxError: invalid syntax (<ipython-input-68-dda3353654b6>, line 3)

#### [10 points] Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve.

In [ ]:
mlp = Sequential()
mlp.add( Dense(input_dim=X_train.shape[1], units=30, activation='relu') )
mlp.add( Dense(units=15, activation='relu') )
mlp.add( Dense(2) )
mlp.add( Activation('softmax') )

mlp.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

mlp.fit(X_train, y_ohe_train, 
        batch_size=32, epochs=100, 
        shuffle=True, verbose=0)

In [ ]:
yhat_mlp = np.argmax(mlp.predict(X_test), axis=1)
ypred_mlp = np.argmax(mlp.predict_proba(X_test), axis=1)
acc_mlp = mt.accuracy_score(y_test,yhat_mlp)

cnn_fpr, cnn_tpr, cnn_thresholds = mt.roc_curve(y_test, yhat_probs, pos_label=0)
cnn_fpr1, cnn_tpr1, cnn_thresholds1 = mt.roc_curve(y_test, yhat_probs, pos_label=1)
cnn_auc_score = mt.roc_auc_score(y_test, yhat_probs)

mlp_fpr, mlp_tpr, mlp_thresholds = mt.roc_curve(y_test, ypred_mlp, pos_label=0)
mlp_fpr1, mlp_tpr1, mlp_thresholds1 = mt.roc_curve(y_test, ypred_mlp, pos_label=1)
mlp_auc_score = mt.roc_auc_score(y_test, ypred_mlp)

## Exceptional Work (10 points total)
#### One idea: Visualize the convolutional filters chosen by your CNN. Try to interpret some of their meanings.